In [126]:
import pandas as pd
from openpyxl.workbook import Workbook
pd.options.display.max_rows=12

In [133]:
#load excel file into pandas dataframe
df = pd.read_excel('AlzheimerData.xlsx', skiprows=1)
df

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,517.06,9.233,1.130,0,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
1,504.06,9.001,1.129,0,8,16,4,0.068,4,0,12,1,4,39.3,1.8,7.1
2,490.03,9.246,1.128,0,8,16,3,0.054,4,0,12,1,4,39.6,1.9,7.5
3,1443.08,6.414,1.146,0,41,20,42,0.187,14,#@@^,6,13,17,32.0,5.8,7.6
4,517.06,9.233,1.130,0,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,533.11,8.74,1.133,0,10,16,4,0.062,4,0,NaN,2,4,37.7,3.3,6.6
21,477.50,8.527,1.128,0,5,21,5,0.083,3,0,18,3,4,46.4,5.4,7.1
22,548.59,8.312,1.132,0,7,22,7,0.100,4,0,18,4,5,43.9,6.1,7.6
23,511.72,7.207,1.145,0,6,9,5,0.067,3,0,6,3,2,36.6,4.2,2.8


In [146]:
#Remove all columns with zero value
df = df.loc[:, (df != 0).any(axis=0)]
df

,Y,X1,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,517.06,9.233,1.130,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
1,504.06,9.001,1.129,8,16,4,0.068,4,0,12,1,4,39.3,1.8,7.1
2,490.03,9.246,1.128,8,16,3,0.054,4,0,12,1,4,39.6,1.9,7.5
3,1443.08,6.414,1.146,41,20,42,0.187,14,#@@^,6,13,17,32.0,5.8,7.6
4,517.06,9.233,1.130,7,16,4,0.067,4,0,12,2,4,39.3,3.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,533.11,8.74,1.133,10,16,4,0.062,4,0,NaN,2,4,37.7,3.3,6.6
21,477.50,8.527,1.128,5,21,5,0.083,3,0,18,3,4,46.4,5.4,7.1
22,548.59,8.312,1.132,7,22,7,0.100,4,0,18,4,5,43.9,6.1,7.6
23,511.72,7.207,1.145,6,9,5,0.067,3,0,6,3,2,36.6,4.2,2.8


In [129]:
#check type of each coulum and then count of coulmn for each type 
df.dtypes.value_counts()

object     6
float64    5
int64      4
dtype: int64

In [147]:
#dfo is the part of dataframe that contain garbage characters and should be clean
dfo=df.select_dtypes('object')

dfo

,X1,X5,X9,X10,X11,X15
0,9.233,16,0,12,2,7.1
1,9.001,16,0,12,1,7.1
2,9.246,16,0,12,1,7.5
3,6.414,20,#@@^,6,13,7.6
4,9.233,16,0,12,2,7.1
...,...,...,...,...,...,...
20,8.74,16,0,NaN,2,6.6
21,8.527,21,0,18,3,7.1
22,8.312,22,0,18,4,7.6
23,7.207,9,0,6,3,2.8


In [148]:
#show total number of each type in dfo(the part that want to clean). 
dfo.applymap(type).stack().value_counts()

<class 'int'>      97
<class 'float'>    47
<class 'str'>       6
dtype: int64

In [159]:
#Parse function: input x(cell of the daraframe), return True if length of x equal and greater than 5 and False if less than 5
# this function is used to check whether we should drop a row or coulmn(if there is a string more than 5, they should be dropped)
def parse(x):
    if type(x)== str:
        import re
        x_str=re.sub(x,'[0-9]','')
        if len(x_str)>=5:
            return  True
    return False

#Remove all rows and columns if they have string more than 5 by the use of pars function to check it
#dfo2 keep rows and columns of dfo that all cells of that row or column are false(for rows sum(1)==0) and for coulumn sum(0)==0)
dfo2= dfo.loc[dfo.applymap(parse).sum(1)==0,dfo.applymap(parse).sum(0)==0]

#Parse_replace: replce each cell of dataframe by float number except for any error( that types are not int or float),
#so replace them (that are string less than 5)by 0
def parse_replace(x):
    try:
        return float(x)
    except:
        return 0

#Concat dfo2(the part of dataframe that should be cleaned and got cleaned) with the rest clean part of data frame
df2=pd.concat([df.select_dtypes(exclude='object'),dfo2.applymap(parse_replace)],1)[df.columns].fillna(0)
df2
              

,Y,X1,X2,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,517.06,9.233,1.130,7,16.0,4,0.067,4,0.0,12.0,2.0,4,39.3,3.6,7.1
1,504.06,9.001,1.129,8,16.0,4,0.068,4,0.0,12.0,1.0,4,39.3,1.8,7.1
2,490.03,9.246,1.128,8,16.0,3,0.054,4,0.0,12.0,1.0,4,39.6,1.9,7.5
3,1443.08,6.414,1.146,41,20.0,42,0.187,14,0.0,6.0,13.0,17,32.0,5.8,7.6
4,517.06,9.233,1.130,7,16.0,4,0.067,4,0.0,12.0,2.0,4,39.3,3.6,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,533.11,8.740,1.133,10,16.0,4,0.062,4,0.0,0.0,2.0,4,37.7,3.3,6.6
21,477.50,8.527,1.128,5,21.0,5,0.083,3,0.0,18.0,3.0,4,46.4,5.4,7.1
22,548.59,8.312,1.132,7,22.0,7,0.100,4,0.0,18.0,4.0,5,43.9,6.1,7.6
23,511.72,7.207,1.145,6,9.0,5,0.067,3,0.0,6.0,3.0,2,36.6,4.2,2.8


In [163]:
#Calulate standardization( transforms data to have a mean of zero and a standard deviation of 1) and normalization(scale a variable to have a values between 0 and 1)
x_standardization=lambda x: (x-x.mean())/x.std()
x_normalization=lambda x: (x-x.mean())/(x.max()-x.min())
#df2.apply(x_standardization).T.dropna(how='all').T

In [164]:
#apply x_standardization to dataframe
df2.loc[:,df2.std()>0].apply(x_standardization)



,Y,X1,X2,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15
0,-0.174260,0.571426,-0.132721,-0.206758,-0.093818,-0.297724,-0.323323,-0.113899,-0.029941,-0.280110,-0.072232,-0.337211,-0.239941,0.543143
1,-0.243789,0.445009,-0.247136,-0.063176,-0.093818,-0.297724,-0.290465,-0.113899,-0.029941,-0.692036,-0.072232,-0.337211,-1.341708,0.543143
2,-0.318826,0.578510,-0.361550,-0.063176,-0.093818,-0.430637,-0.750478,-0.113899,-0.029941,-0.692036,-0.072232,-0.260803,-1.280499,0.752851
3,4.778422,-0.964648,1.697914,4.675022,0.687995,4.752955,3.619646,4.631891,-1.152728,4.251079,4.622817,-2.196453,1.106665,0.805278
4,-0.174260,0.571426,-0.132721,-0.206758,-0.093818,-0.297724,-0.323323,-0.113899,-0.029941,-0.280110,-0.072232,-0.337211,-0.239941,0.543143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,-0.088419,0.302790,0.210523,0.223988,-0.093818,-0.297724,-0.487614,-0.113899,-2.275516,-0.280110,-0.072232,-0.744716,-0.423569,0.281009
21,-0.385841,0.186726,-0.361550,-0.493921,0.883449,-0.164812,0.202406,-0.588478,1.092846,0.131816,-0.072232,1.471094,0.861827,0.543143
22,-0.005626,0.069573,0.096108,-0.206758,1.078902,0.101014,0.760993,-0.113899,1.092846,0.543743,0.288926,0.834367,1.290293,0.805278
23,-0.202820,-0.532542,1.583499,-0.350340,-1.461990,-0.164812,-0.323323,-0.588478,-1.152728,0.131816,-0.794547,-1.024876,0.127315,-1.711216


In [165]:
#apply x_normalization to the dataframe
df2.loc[:,df2.std()>0].apply(x_normalization)

,Y,X1,X2,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15
0,-0.033743,0.111054,-0.026977,-0.037895,-0.020870,-0.057436,-0.052620,-0.021818,-0.008889,-0.052308,-0.013333,-0.085419,-0.058507,0.131139
1,-0.047207,0.086485,-0.050233,-0.011579,-0.020870,-0.057436,-0.047273,-0.021818,-0.008889,-0.129231,-0.013333,-0.085419,-0.327164,0.131139
2,-0.061737,0.112430,-0.073488,-0.011579,-0.020870,-0.083077,-0.122139,-0.021818,-0.008889,-0.129231,-0.013333,-0.066065,-0.312239,0.181772
3,0.925286,-0.187474,0.345116,0.856842,0.153043,0.916923,0.589091,0.887273,-0.342222,0.793846,0.853333,-0.556387,0.269851,0.194430
4,-0.033743,0.111054,-0.026977,-0.037895,-0.020870,-0.057436,-0.052620,-0.021818,-0.008889,-0.052308,-0.013333,-0.085419,-0.058507,0.131139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,-0.017121,0.058846,0.042791,0.041053,-0.020870,-0.057436,-0.079358,-0.021818,-0.675556,-0.052308,-0.013333,-0.188645,-0.103284,0.067848
21,-0.074714,0.036289,-0.073488,-0.090526,0.196522,-0.031795,0.032941,-0.112727,0.324444,0.024615,-0.013333,0.372645,0.210149,0.131139
22,-0.001090,0.013521,0.019535,-0.037895,0.240000,0.019487,0.123850,-0.021818,0.324444,0.101538,0.053333,0.211355,0.314627,0.194430
23,-0.039274,-0.103497,0.321860,-0.064211,-0.325217,-0.031795,-0.052620,-0.112727,-0.342222,0.024615,-0.146667,-0.259613,0.031045,-0.413165
